In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from tabulate import tabulate 
import lightgbm as lgb


train_data = pd.read_csv('/kaggle/input/finaldf/final_df.csv')

train_data.head()
print(train_data)

            id  num_backspaces  duration_backspaces  num_cut/copy/paste  \
0     001519c8           417.0              34130.0                 417   
1     0022f953           260.0              23550.0                 261   
2     0042269b           439.0              32905.0                 439   
3     0059420b           152.0              18411.0                 152   
4     0075873a           517.0              40199.0                 517   
...        ...             ...                  ...                 ...   
2466  911a0eed             0.0                  0.0                 285   
2467  97bb1cdc             0.0                  0.0                 142   
2468  ace0ee61             0.0                  0.0                  26   
2469  be2130ad             0.0                  0.0                   2   
2470  d6e700e9             0.0                  0.0                  43   

      discarded_text  D/I Ratio  Proportion_of_Deletions  no_distant_revision  \
0                4

In [2]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_data.drop(['score', 'id'], axis=1),
    train_data['score'],
    test_size=0.2,
    random_state=42
)
X_train.head()
y_train.head()

1610    4.5
1525    5.0
2366    2.5
859     3.0
497     2.0
Name: score, dtype: float64

In [3]:
best_params = {'boosting_type': 'gbdt', 
               'metric': 'rmse',
               'bagging_freq': 1,
               'bagging_fraction': 0.75,
               'learning_rate': 0.05, 
               'num_leaves': 19, 
               'min_child_samples': 46,
               'verbosity': -1,
               'random_state': 42,
               'n_estimators': 500,
               'device_type': 'cpu'}

params = {
            "objective": "regression",
            "metric": "rmse",
            'random_state': 42,
            "n_estimators" : 12001,
            "verbosity": -1,
            "device_type": "cpu",
            **best_params
}


model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print(f"Root Mean Squared Error: {rmse}")

score_table = pd.DataFrame({'Expected Score': y_valid, 'Actual Score': y_pred})
print(tabulate(score_table, headers='keys', tablefmt='psql'))

[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75
Root Mean Squared Error: 0.6791570479283168
+------+------------------+----------------+
|      |   Expected Score |   Actual Score |
|------+------------------+----------------|
| 1058 |              5   |        4.86629 |
| 2178 |              1.5 |        2.32478 |
| 1708 |              3   |        2.42397 |
| 1550 |              3   |        3.8706  |
|  353 |              3.5 |        3.04705 |
| 1017 |              3.5 |        4.00268 |
|  438 |              4   |        4.51021 |
| 1694 |              2.5 |        3.08161 |
|  259 |              4.5 |        4.51092 |
| 1971 |              4   |        4.41863 |
| 1922 |              4.5 |        4.28943 |
| 1736 |              5   |        4.32965 |
| 1777 |              3   |        2.38235 |
| 2043 | 